# IMPORT STATEMENTS

In [1]:
!TMPDIR=/home/mgill/ pip install --cache-dir=/home/mgill/ --build /home/mgill/ pandas numpy sklearn xgboost matplotlib tensorflow==2.1.0 keras==2.2.4 fastai python-dateutil scikit-optimize scikit-learn==0.21 graphviz

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.regularizers import *
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from fastai.tabular.all import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from random import randint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Activation
from math import sqrt
from statistics import mean
from sklearn.metrics import mean_squared_error, r2_score
import skopt
from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import GridSearchCV
import pickle
from collections import OrderedDict
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from sklearn.impute import SimpleImputer

2.1.0
2.2.4


Using TensorFlow backend.


In [3]:
def new_prep_data(tt_file, ho_file):
    imp = SimpleImputer(missing_values='./.', strategy='most_frequent')
    my_list = []
    x = 0 
    for chunk in pd.read_csv(tt_file, chunksize=10000, index_col="Unnamed: 0"):
        x=x+10000
        chunk = chunk.T
        if 'Value' in chunk.columns:
            #does the selecting of pheno array for application ML
            chunk["Value"] = pd.to_numeric(chunk["Value"], downcast="float")
            tt_pheno = chunk["Value"].to_numpy()
            #reshapes it so its not a 1D array
            print(tt_pheno.shape)
            tt_pheno = np.reshape(tt_pheno,(len(tt_pheno),1))
            print(tt_pheno.shape)
            chunk = chunk.drop(columns=['Value'])
        headers = chunk.columns
        row_idx = chunk.index
        chunk = imp.fit_transform(chunk) #SHOULD TURN ./. into the most common for each column
        #since imputing makes a numpy array have to turn back into PD for label encoding
        chunk = pd.DataFrame(data = chunk, index = row_idx, columns = headers)
        my_list.append(chunk)
        print(x)
    tt_vcf = pd.concat(my_list, axis = 1)
    my_list = []
    x=0
    for chunk in pd.read_csv(ho_file, chunksize=10000, index_col="Unnamed: 0"):
        x=x+10000
        chunk = chunk.T
        if 'Value' in chunk.columns:
            #does the selecting of pheno array for application ML
            chunk["Value"] = pd.to_numeric(chunk["Value"], downcast="float")
            ho_pheno = chunk["Value"].to_numpy()
            #reshapes it so its not a 1D array
            print(ho_pheno.shape)
            ho_pheno = np.reshape(ho_pheno,(len(ho_pheno),1))
            print(ho_pheno.shape)
            chunk = chunk.drop(columns=['Value'])
        headers = chunk.columns
        row_idx = chunk.index
        chunk = imp.fit_transform(chunk) #SHOULD TURN ./. into the most common for each column
        #since imputing makes a numpy array have to turn back into PD for label encoding
        chunk = pd.DataFrame(data = chunk, index = row_idx, columns = headers)
        my_list.append(chunk)
        print(x)
    ho_vcf = pd.concat(my_list, axis = 1)
    print(tt_vcf.shape)
    print(ho_vcf.shape)
    return tt_vcf, ho_vcf, tt_pheno, ho_pheno

In [ ]:
tt_vcf, ho_vcf, tt_pheno, ho_pheno = new_prep_data("SCC_Merged_filtered.csv_train_testQTL_SNPS.csv", "SCC_Merged_filtered.csv_holdoutQTL_SNPS.csv")

In [ ]:
#if it hasn't been run and saved before
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe = ohe.fit(tt_vcf)

In [ ]:
pickle.dump(ohe, open("SCC_QTL_ohe.dat", "wb"))

In [ ]:
#if need or have new holdout data etc.
ohe = pickle.load(open("SCC_QTL_ohe.dat", "rb"))

In [ ]:
print(tt_vcf.shape)
tt_vcf = ohe.transform(tt_vcf)
print(tt_vcf.shape)
print(ho_vcf.shape)
ho_vcf = ohe.transform(ho_vcf)
print(ho_vcf.shape)

In [ ]:
def find_snp_from_header(ohe,snp_num):
    count = 0
    snp = "Not found"
    found = False
    i = 0
    while i < len(ohe.categories_) and (found == False):
        j = 0
        while j < len(ohe.categories_[i]):
            if(count == snp_num):
                snp = ohe.categories_[i][j]
                found = True
                break
            count = count + 1
            j = j + 1
        i = i + 1
    return snp

In [ ]:
## TESTING IF IT WORKS
my_snp = find_snp_from_header(ohe, 462793)
print(my_snp)

# XGBOOST ML Model

In [ ]:
print(tt_vcf.shape)
print(tt_pheno.shape)
seed = randint(0,5000)
X_train, X_test, y_train, y_test = train_test_split(tt_vcf, tt_pheno, test_size=0.2, random_state=seed)
print(X_test.shape)
print("seed is " + str(seed))

In [ ]:
space ={'learning_rate': Real(0.01, 1.0, 'log-uniform'),
        'min_child_weight': Integer(0, 10),
        'max_depth': Integer(0, 50),
        'max_delta_step': Integer(0, 20),
        'subsample': Real(0.01, 1.0, 'uniform'),
        'colsample_bytree': Real(0.01, 1.0, 'uniform'),
        'colsample_bylevel': Real(0.01, 1.0, 'uniform'),
        'reg_lambda': Real(1e-9, 1000, 'log-uniform'),
        'reg_alpha': Real(1e-9, 1.0, 'log-uniform'),
        'gamma': Real(1e-9, 0.5, 'log-uniform'),
        'min_child_weight': Integer(0, 5),
        'n_estimators': Integer(50, 200),
        'scale_pos_weight': Real(1e-6, 500, 'log-uniform')}

In [ ]:
def on_step(optim_result):
    """
    Callback meant to view scores after
    each iteration while performing Bayesian
    Optimization in Skopt"""
    score = xgb_bayes_search.best_score_
    print("best score: %s" % score)
    if score >= 0.98:
        print('Interrupting!')
        return True

In [ ]:
xgbcl = xgb.XGBClassifier(objective='multi:softmax', num_class=4)
xgb_bayes_search = BayesSearchCV(xgbcl, space, n_iter=32, # specify how many iterations
                                    scoring=None, n_jobs=1, cv=5, verbose=3, random_state=42, n_points=12,
                                 refit=True)
xgb_bayes_search.fit(X_train, y_train.ravel(), callback = on_step)
print(xgb_bayes_search.best_params_)

In [ ]:
print(xgb_bayes_search.best_params_)
model = xgb.XGBClassifier(**xgb_bayes_search.best_params_, objective='multi:softmax', num_class=4)

In [ ]:
##if not in same run as optimisation
best_params = OrderedDict([('colsample_bylevel', 0.25617325301227906), ('colsample_bytree', 0.7083937150495909), ('gamma', 2.41812432168581e-07), ('learning_rate', 0.13965555720269418), ('max_delta_step', 10), ('max_depth', 27), ('min_child_weight', 1), ('n_estimators', 76), ('reg_alpha', 3.178148842971562e-08), ('reg_lambda', 0.005381781269387993), ('scale_pos_weight', 0.23835043249575294), ('subsample', 0.9559763235078597)])
model = xgb.XGBClassifier(**best_params, objective='multi:softmax', num_class=4)

In [ ]:
## DO MORE STUFF HERE when OPTIMISED

In [ ]:
def eval_k_fold(m, x, y, k, hx, hy):
    #model: xgboost model, should be with the best params available
    #x: input data (eg. all samples and SNPS)
    #y: labels
    #k: number of folds for cross validation
    cv = StratifiedKFold(n_splits=k,shuffle=False)
    fig1 = plt.figure(figsize=[12,12])

    tprs = []
    aucs = []
    results = []
    mean_fpr = np.linspace(0,1,100)
    high = 0
    best = m
    i = 1
    for train,test in cv.split(x,y):
        prediction = m.fit(x[train],y[train].ravel()).predict_proba(x[test])
        print("variables for auroc curve done. Processing fold accuracy + checking best model")
        y_pred = m.predict(x[test])
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y[test], predictions)
        pcent = accuracy * 100.0
        print("The accuracy of this model is" + str(pcent))
        if(pcent > high):
            high = pcent
            best = m
       # fpr, tpr, t = roc_curve(y[test], prediction[:, 1])
       # tprs.append(interp(mean_fpr, fpr, tpr))
       # roc_auc = auc(fpr, tpr)
       # aucs.append(roc_auc)
        results.append(pcent)
       # plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
        i= i+1

   # plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
   # mean_tpr = np.mean(tprs, axis=0)
   # mean_auc = auc(mean_fpr, mean_tpr)
   # plt.plot(mean_fpr, mean_tpr, color='blue',
    #         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)
    
    holdout_pred = best.predict(hx)
    predictions = [round(value) for value in holdout_pred]
    #sees how accurate the model was when testing the test set
    accuracy = accuracy_score(hy, predictions)
    pcent = accuracy * 100.0
    print(pcent)
    xgb_predictions = best.predict(hx)
    xgb_probs = best.predict_proba(hx)[:, 1]
    #model_fpr, model_tpr, my_roccy = evaluate_model(xgb_predictions, xgb_probs, hy)
    #plt.plot(model_fpr, model_tpr, 'r', label = 'Holdout Data'+my_roccy, lw=2)
    
    #plt.xlabel('False Positive Rate')
    #plt.ylabel('True Positive Rate')
    #plt.title('ROC Flower Colour Training Model & Holdout Data')
    #plt.legend(loc="lower right")
    #plt.show()

    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    print("Holdout Accuracy: %.2f%%" % (pcent))
    
    return best

In [ ]:
print(tt_vcf.shape)
print(tt_pheno.shape)
print(ho_vcf.shape)
print(ho_pheno.shape)
 #if optimised in same session, other enter manually below
#this function should average out 10 folds and training, with inital params optimised
#average accuracy and std should be calculated along with a nice AUROC graph of train/test models
#best model should be extracted for use on holdout set
best_model = eval_k_fold(model, tt_vcf, tt_pheno, 10, ho_vcf, ho_pheno)

In [ ]:
import pickle
pickle.dump(best_model, open("SCC_kfold_10_XGB_QTL.pickle.dat", "wb"))

In [ ]:
#only load if not generated in same session
best_model = pickle.load(open("SCC_kfold_10_XGB_QTL.pickle.dat", "rb"))

In [ ]:
## DO SNPS of importance

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot
#best_model = pickle.load(open("FC_kfold_10_tt_from_all.pickle.dat", "rb"))
plt.figure(figsize = (20, 20))
plot_importance(best_model, max_num_features=15, importance_type='gain', height=0.3)
pyplot.show()

In [ ]:
#This function essentially returns an array of dataframe headers the length of OHE'd input SNPs for training data
#EG. It will be able to determine that feature 357310 is Gm13_17683957 but not what allele it is
#eg. feature 357309 357310 and 357311 may all be one hot encoded versions of all possible values of Gm13_17683957
#iterating through the saved OHE will by able to determine what specific allele the feature is but cannot determine
#what SNP header it belongs to. Therefore combining these two methods you can determine both allele and SNP
snp = []
imp = SimpleImputer(missing_values='./.', strategy='most_frequent')
fs_ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
x = 0
n_headers = []
le = LabelEncoder()
#while (i < 10):
for chunk in pd.read_csv("SCC_Merged_filtered.csv_train_testQTL_SNPS.csv", chunksize=10000, index_col="Unnamed: 0"):
    chunk = chunk.T
    if 'Value' in chunk.columns:
        print("dropping value so it doesn't include that in headers")
        chunk = chunk.drop(columns=['Value'])
    headers = chunk.columns
    row_idx = chunk.index
    chunk = imp.fit_transform(chunk) #SHOULD TURN ./. into the most common for each column
    #since imputing makes a numpy array have to turn back into PD for label encoding
    chunk = pd.DataFrame(data = chunk, index = row_idx, columns = headers)
    chunk = chunk.apply(lambda col: le.fit_transform(col))
    c_headers = chunk.columns
    y = 0
    for column in chunk:
        d = (chunk[column].nunique())
        n_headers.extend([c_headers[y] for i in range(d)])
        #print(n_headers)
        #print(l)
        #n_headers.append(c_headers[y] * d)
        #print(n_headers)
        y = y + 1
    #to double check that it would indeed be one hot encoded with this amount of columns
    chunk = fs_ohe.fit_transform(chunk)
    x = x + chunk.shape[1]
    print("my X value is: " + str(x))
    print(chunk.shape)
    print("my header list is: " + str(len(n_headers)))
print(len(n_headers))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k')
fs = [181106,56313,581117,214419,133458,214671,214582,214543,213998,214557,214545,214000,331158,30222,214531]
scores = [125,92.7,57.8,34.9,27.5,22.4,20.3,20.2,19.9,16.9,16.8,16.1,15.51,15.5,13.1]
snp_label = []
for jj in fs:
    jj_allele = find_snp_from_header(ohe, jj)
    this_snp = (n_headers[jj] + ' ('+str(jj_allele)+')')
    print(this_snp)
    snp_label.append(this_snp)
snp_label.reverse()
scores.reverse()
print(len(scores))
print(len(snp_label))
plt.barh(snp_label,scores)
plt.title('SNP Importance XGBoost Seed Coat Colour')
plt.ylabel('SNP Label')
plt.xlabel('Relative F_Score (GAIN)')
plt.show()

In [ ]:
my_dict = best_model.get_booster().get_score(importance_type="gain")

In [ ]:
def rename_f_header(fn,n_headers,ohe):
    fn = fn[1:]
    fn = int(fn)
    allele = find_snp_from_header(ohe, fn)
    this_snp = (n_headers[fn] + ' ('+str(allele)+')')
    return this_snp

In [ ]:
#convert feature to actual SNP name
i = 0
new_dict = {}
for key in my_dict:
    new_key = rename_f_header(key, n_headers, ohe)
    new_dict[new_key] = my_dict[key]
    i = i + 1
    print(str(i))
    if(my_dict):
        continue
    else:
        break

In [ ]:
new_fi = pd.Series(new_dict)
print(new_fi)
df = new_fi.to_frame()
df = df.rename(columns = {0:'F_Score(GAIN)'})
print(df)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k')
indexes = df.nlargest(20, "F_Score(GAIN)").index
values = df.nlargest(20, "F_Score(GAIN)").values.ravel()
indexes = indexes[::-1]
values = values[::-1]
plt.barh(indexes, values)
plt.title('SNP Importance XGBoost Seed Coat Colour')
plt.ylabel('SNP Label')
plt.xlabel('Relative F_Score (GAIN)')
plt.show()

In [ ]:
#generate figure object
figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k')
#load in the 20 lardest values and their SNP label
indexes = df.nlargest(20, "F_Score(GAIN)").index
values = df.nlargest(20, "F_Score(GAIN)").values.ravel()
#reverse to make the largest be at the front
indexes = indexes[::-1]
values = values[::-1]
#for each different chromosome you want to colour add a index(*_i) and value (*_v) array
#black would be colour for singular/notinteresting chromosomes
r_i = []
r_v = []
b_i = []
b_v = []
g_i = []
g_v = []
y_i = []
y_v = []
bl_i = []
bl_v = []
p_i = []
p_v = []
br_i = []
br_v = []
pu_i = []
pu_v = []
#for each value in the top n (default 20) check which chromosome it belongs to and add it to the colour array
i = 0
while i < len(indexes):
    if('Gm12' in indexes[i]):
        r_i.append(indexes[i])
        r_v.append(values[i])
    elif('Gm08' in indexes[i]):
        b_i.append(indexes[i])
        b_v.append(values[i])
    elif('Gm06' in indexes[i]):
        g_i.append(indexes[i])
        g_v.append(values[i])
    elif('Gm02' in indexes[i]):
        y_i.append(indexes[i])
        y_v.append(values[i])
    elif('Gm19' in indexes[i]):
        p_i.append(indexes[i])
        p_v.append(values[i])
   # elif('Gm04' in indexes[i]):
   #     br_i.append(indexes[i])
   #     br_v.append(values[i])
   # elif('Gm13' in indexes[i]):
   #     pu_i.append(indexes[i])
   #     pu_v.append(values[i])
    else:
        bl_i.append(indexes[i])
        bl_v.append(values[i])
    i = i + 1
#plot each of the arrays with appropriate colour and label graph
plt.barh(bl_i, bl_v, color="black")
plt.barh(br_i, br_v, color="brown")
plt.barh(pu_i, pu_v, color="purple")
plt.barh(p_i, p_v, color="orange")
plt.barh(y_i, y_v, color="yellow")
plt.barh(g_i, g_v, color="green")
plt.barh(r_i, r_v, color="red")
plt.barh(b_i, b_v, color="blue")
plt.title('SNP Importance XGBoost Seed Coat Colour')
plt.ylabel('SNP Label')
plt.xlabel('Relative F_Score (GAIN)')
plt.show()

# RL Model

In [ ]:
tt_vcf, ho_vcf, tt_pheno, ho_pheno = new_prep_data("SCC_Merged_filtered.csv_train_testQTL_SNPS.csv", "SCC_Merged_filtered.csv_holdoutQTL_SNPS.csv")

In [ ]:
#if need or have new holdout data etc.
ohe = pickle.load(open("SCC_QTL_ohe.dat", "rb"))

In [ ]:
print(tt_vcf.shape)
tt_vcf = ohe.transform(tt_vcf)
print(tt_vcf.shape)
print(ho_vcf.shape)
ho_vcf = ohe.transform(ho_vcf)
print(ho_vcf.shape)

In [ ]:
print(tt_vcf.shape)
print(tt_pheno.shape)
print(ho_vcf.shape)
print(ho_pheno.shape)
seed = randint(0,5000)
 #if optimised in same session, other enter manually below
#this function should average out 10 folds and training, with inital params optimised
#average accuracy and std should be calculated along with a nice AUROC graph of train/test models
#best model should be extracted for use on holdout set
model = RandomForestClassifier(n_estimators=100, random_state=seed, max_features = 'sqrt',n_jobs=1, verbose = 1)
best_model = eval_k_fold(model, tt_vcf, tt_pheno, 10, ho_vcf, ho_pheno)

# CNN (based off primer paper and Philipp)

In [ ]:
tt_vcf, ho_vcf, tt_pheno, ho_pheno = new_prep_data("SCC_Merged_filtered.csv_train_testQTL_SNPS.csv", "SCC_Merged_filtered.csv_holdoutQTL_SNPS.csv")

In [ ]:
ohe = pickle.load(open("SCC_QTL_ohe.dat", "rb"))
tt_vcf = ohe.transform(tt_vcf)
print(tt_vcf.shape)
print(ho_vcf.shape)
ho_vcf = ohe.transform(ho_vcf)
print(ho_vcf.shape)

In [ ]:
##how to mlb both tt and ho for same scheme? do i even need to?

mlb = MultiLabelBinarizer()
mlb = mlb.fit(tt_pheno)
##print(tt_pheno.shape)
#print(ho_pheno.shape)
#tt_pheno = mlb.transform(tt_pheno)
#print(tt_pheno.shape)
#ho_pheno = mlb.transform(ho_pheno)
#print(ho_pheno.shape)

In [ ]:
def build_CNN_model(x_len):    
    #del model
    model = Sequential()
    model.add(Conv1D(filters=10, kernel_size=10, 
                     input_shape=(x_len, 1)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(filters=8, kernel_size=8, 
                     input_shape=(10, 1)))
    model.add(Activation('linear'))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=6, kernel_size=6, 
                     input_shape=(8, 1)))
    model.add(Activation('linear'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(24, activation='linear'))
    model.add(Dense(16, activation='linear'))
    model.add(Dense(8, activation='linear'))
    model.add(BatchNormalization())
    model.add(Dense(4, activation='softmax'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    print(model.summary())
    return model

In [ ]:
def eval_cnn(x,y,k,mlb):
    cv = StratifiedKFold(n_splits=k,shuffle=False)
    best_model = []
    results = []
    highest = 0
    i = 1
    for train,test in cv.split(x,y):
        print(y.shape)
        print(y[train])
        if(i==1):
            y = mlb.transform(y)
            print(y.shape)
            print(y[train])
        x = x.reshape(x.shape[0], x.shape[1], 1)
        model = build_CNN_model(x[train].shape[1])
        bs = ((x[train].shape[0])/20)
        bs = round(bs)
        history = model.fit(x[train], y[train], validation_data=(x[test], y[test]), epochs=100, batch_size=bs)
        _, accuracy = model.evaluate(x[test], y[test], batch_size=bs, verbose=0)
        accuracy = accuracy *100
        print("accuracy for model " + str(i) + " is " + str(accuracy))
        if(accuracy > highest):
            highest = accuracy
            best_model = model
        results.append(accuracy)
        del model
        i = i + 1
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results))) 
    return best_model

In [ ]:
best_CNN = eval_cnn(tt_vcf, tt_pheno, 10, mlb)
#import pickle
#pickle.dump(best_CNN, open("SCC_CNN_model.pickle.dat", "wb"))

In [ ]:
bs = ((ho_vcf.shape[0])/40)
bs = round(bs)
ho_vcf = ho_vcf.reshape(ho_vcf.shape[0], ho_vcf.shape[1],1)
ho_pheno = mlb.transform(ho_pheno)
_, accuracy = best_CNN.evaluate(ho_vcf, ho_pheno, batch_size=bs, verbose=0)
print("Holdout accuracy is " + str(accuracy*100))

In [ ]:
accuracy

In [ ]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
vcf = vcf.reshape(vcf.shape[0], vcf.shape[1], 1)
pheno = pheno.reshape(pheno.shape[0], pheno.shape[1])
estimator = KerasClassifier(build_fn=CNN_model, epochs=10, batch_size=32, verbose=1)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, vcf, pheno, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

# DNN (based off yield prediction paper)

In [4]:
tt_vcf, ho_vcf, tt_pheno, ho_pheno = new_prep_data("SCC_Merged_filtered.csv_train_testQTL_SNPS.csv", "SCC_Merged_filtered.csv_holdoutQTL_SNPS.csv")

10000
20000
30000
40000
(620,)
(620, 1)
50000
10000
20000
30000
40000
(155,)
(155, 1)
50000
(620, 43568)
(155, 43568)


In [5]:
ohe = pickle.load(open("SCC_QTL_ohe.dat", "rb"))
tt_vcf = ohe.transform(tt_vcf)
print(tt_vcf.shape)
print(ho_vcf.shape)
ho_vcf = ohe.transform(ho_vcf)
print(ho_vcf.shape)

(620, 125766)
(155, 43568)
(155, 125766)


In [6]:
##how to mlb both tt and ho for same scheme? do i even need to?

mlb = MultiLabelBinarizer()
mlb = mlb.fit(tt_pheno)
##print(tt_pheno.shape)
#print(ho_pheno.shape)
#tt_pheno = mlb.transform(tt_pheno)
#print(tt_pheno.shape)
#ho_pheno = mlb.transform(ho_pheno)
#print(ho_pheno.shape)

In [7]:
#My own DNN model based upon paper
#del model #incase its stored a previous model
#del history #for redoing shit

#do batch size as 64
#reduce the inputs by half when you read it in
#add XGboost and RF to the one notebook
def build_DNN_model(x_len):
    model = Sequential()

    #add first input layer, with no normalization
    model.add(Dense(192, input_dim = x_len))
    model.add(Activation('relu'))
    model.add(Dropout(0.03))
    
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.02))
    
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.01))
    
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(16))
    model.add(Activation('relu'))
    #add output layer
    model.add(Dense(4, activation='softmax'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    print(model.summary())
    return model

In [8]:
def eval_dnn(x,y,k,mlb):
    cv = StratifiedKFold(n_splits=k,shuffle=False)
    best_model = []
    results = []
    highest = 0
    i = 1
    for train,test in cv.split(x,y):
        print(y.shape)
        print(y[train])
        if(i==1):
            y = mlb.transform(y)
            print(y.shape)
            print(y[train])
        model = build_DNN_model(x[train].shape[1])
        bs = ((x[train].shape[0])/20)
        bs = round(bs)
        history = model.fit(x[train], y[train], validation_data=(x[test], y[test]), epochs=100, batch_size=bs)
        _, accuracy = model.evaluate(x[test], y[test], batch_size=bs, verbose=0)
        accuracy = accuracy *100
        print("accuracy for model " + str(i) + " is " + str(accuracy))
        if(accuracy > highest):
            highest = accuracy
            best_model = model
        results.append(accuracy)
        del model
        i = i + 1
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results))) 
    return best_model

In [9]:
best_DNN = eval_dnn(tt_vcf, tt_pheno, 10, mlb)

(620, 1)
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [0.]
 [2.]
 [2.]
 [1.]
 [3.]
 [0.]
 [2.]
 [1.]
 [0.]
 [2.]
 [2.]
 [2.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [0.]
 [2.]
 [2.]
 [2.]
 [3.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [2.]
 [2.]
 [3.]
 [0.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [1.]
 [2.]
 [3.]
 [2.]
 [2.]
 [2.]
 [2.]
 [1.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [3.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 

556/556 [==============================] - 0s 664us/sample - loss: 0.2933 - accuracy: 0.8921 - val_loss: 0.5690 - val_accuracy: 0.8281
Epoch 18/100
556/556 [==============================] - 0s 662us/sample - loss: 0.3048 - accuracy: 0.8813 - val_loss: 0.7487 - val_accuracy: 0.7344
Epoch 19/100
556/556 [==============================] - 0s 661us/sample - loss: 0.2883 - accuracy: 0.8903 - val_loss: 0.5405 - val_accuracy: 0.8438
Epoch 20/100
556/556 [==============================] - 0s 665us/sample - loss: 0.2712 - accuracy: 0.8957 - val_loss: 0.4739 - val_accuracy: 0.8594
Epoch 21/100
556/556 [==============================] - 0s 662us/sample - loss: 0.2445 - accuracy: 0.9173 - val_loss: 0.5128 - val_accuracy: 0.8438
Epoch 22/100
556/556 [==============================] - 0s 662us/sample - loss: 0.2615 - accuracy: 0.9047 - val_loss: 0.4915 - val_accuracy: 0.8594
Epoch 23/100
556/556 [==============================] - 0s 661us/sample - loss: 0.2168 - accuracy: 0.9137 - val_loss: 0.5133 

556/556 [==============================] - 0s 664us/sample - loss: 0.0176 - accuracy: 0.9946 - val_loss: 1.3295 - val_accuracy: 0.8281
Epoch 73/100
556/556 [==============================] - 0s 660us/sample - loss: 0.0368 - accuracy: 0.9892 - val_loss: 1.0318 - val_accuracy: 0.8438
Epoch 74/100
556/556 [==============================] - 0s 662us/sample - loss: 0.0129 - accuracy: 0.9964 - val_loss: 0.9663 - val_accuracy: 0.8594
Epoch 75/100
556/556 [==============================] - 0s 660us/sample - loss: 0.0156 - accuracy: 0.9946 - val_loss: 0.8654 - val_accuracy: 0.8594
Epoch 76/100
556/556 [==============================] - 0s 663us/sample - loss: 0.0140 - accuracy: 0.9982 - val_loss: 1.1896 - val_accuracy: 0.8281
Epoch 77/100
556/556 [==============================] - 0s 658us/sample - loss: 0.0222 - accuracy: 0.9964 - val_loss: 0.8920 - val_accuracy: 0.8281
Epoch 78/100
556/556 [==============================] - 0s 662us/sample - loss: 0.0148 - accuracy: 0.9964 - val_loss: 1.1561 

556/556 [==============================] - 0s 690us/sample - loss: 0.3528 - accuracy: 0.8687 - val_loss: 0.5072 - val_accuracy: 0.8281
Epoch 11/100
556/556 [==============================] - 0s 684us/sample - loss: 0.3569 - accuracy: 0.8687 - val_loss: 0.7423 - val_accuracy: 0.7969
Epoch 12/100
556/556 [==============================] - 0s 687us/sample - loss: 0.3185 - accuracy: 0.8813 - val_loss: 0.5575 - val_accuracy: 0.7969
Epoch 13/100
556/556 [==============================] - 0s 706us/sample - loss: 0.3241 - accuracy: 0.8777 - val_loss: 0.5018 - val_accuracy: 0.8438
Epoch 14/100
556/556 [==============================] - 0s 684us/sample - loss: 0.3087 - accuracy: 0.8849 - val_loss: 0.5151 - val_accuracy: 0.8438
Epoch 15/100
556/556 [==============================] - 0s 697us/sample - loss: 0.2710 - accuracy: 0.8939 - val_loss: 0.6523 - val_accuracy: 0.7969
Epoch 16/100
556/556 [==============================] - 0s 690us/sample - loss: 0.2679 - accuracy: 0.8903 - val_loss: 1.0092 

556/556 [==============================] - 0s 683us/sample - loss: 0.0166 - accuracy: 0.9982 - val_loss: 0.8367 - val_accuracy: 0.8281
Epoch 66/100
556/556 [==============================] - 0s 679us/sample - loss: 0.0239 - accuracy: 0.9910 - val_loss: 0.9738 - val_accuracy: 0.8125
Epoch 67/100
556/556 [==============================] - 0s 686us/sample - loss: 0.0177 - accuracy: 0.9946 - val_loss: 1.1550 - val_accuracy: 0.7656
Epoch 68/100
556/556 [==============================] - 0s 669us/sample - loss: 0.0164 - accuracy: 0.9946 - val_loss: 0.6174 - val_accuracy: 0.8438
Epoch 69/100
556/556 [==============================] - 0s 719us/sample - loss: 0.0220 - accuracy: 0.9910 - val_loss: 0.9576 - val_accuracy: 0.7812
Epoch 70/100
556/556 [==============================] - 0s 669us/sample - loss: 0.0121 - accuracy: 0.9964 - val_loss: 1.0086 - val_accuracy: 0.7812
Epoch 71/100
556/556 [==============================] - 0s 671us/sample - loss: 0.0124 - accuracy: 0.9982 - val_loss: 0.8638 

...

560/560 [==============================] - 0s 639us/sample - loss: 0.0290 - accuracy: 0.9929 - val_loss: 1.0678 - val_accuracy: 0.7500
Epoch 57/100
560/560 [==============================] - 0s 638us/sample - loss: 0.0159 - accuracy: 0.9964 - val_loss: 1.2158 - val_accuracy: 0.7500
Epoch 58/100
560/560 [==============================] - 0s 635us/sample - loss: 0.0244 - accuracy: 0.9893 - val_loss: 1.2466 - val_accuracy: 0.7833
Epoch 59/100
560/560 [==============================] - 0s 640us/sample - loss: 0.0284 - accuracy: 0.9911 - val_loss: 1.4963 - val_accuracy: 0.7000
Epoch 60/100
560/560 [==============================] - 0s 644us/sample - loss: 0.0154 - accuracy: 0.9946 - val_loss: 1.5005 - val_accuracy: 0.8000
Epoch 61/100
560/560 [==============================] - 0s 639us/sample - loss: 0.0312 - accuracy: 0.9911 - val_loss: 1.2827 - val_accuracy: 0.7833
Epoch 62/100
560/560 [==============================] - 0s 641us/sample - loss: 0.0319 - accuracy: 0.9875 - val_loss: 1.2679 

Train on 560 samples, validate on 60 samples
Epoch 1/100
560/560 [==============================] - 1s 1ms/sample - loss: 0.8390 - accuracy: 0.7232 - val_loss: 3.0271 - val_accuracy: 0.7000
Epoch 2/100
560/560 [==============================] - 0s 634us/sample - loss: 0.5978 - accuracy: 0.8214 - val_loss: 2.8419 - val_accuracy: 0.7000
Epoch 3/100
560/560 [==============================] - 0s 633us/sample - loss: 0.5343 - accuracy: 0.8446 - val_loss: 1.3479 - val_accuracy: 0.7000
Epoch 4/100
560/560 [==============================] - 0s 633us/sample - loss: 0.4849 - accuracy: 0.8589 - val_loss: 0.9569 - val_accuracy: 0.7167
Epoch 5/100
560/560 [==============================] - 0s 632us/sample - loss: 0.4740 - accuracy: 0.8554 - val_loss: 0.9626 - val_accuracy: 0.7167
Epoch 6/100
560/560 [==============================] - 0s 634us/sample - loss: 0.4341 - accuracy: 0.8679 - val_loss: 0.7232 - val_accuracy: 0.7833
Epoch 7/100
560/560 [==============================] - 0s 636us/sample - lo

560/560 [==============================] - 0s 634us/sample - loss: 0.0247 - accuracy: 0.9911 - val_loss: 1.4667 - val_accuracy: 0.7333
Epoch 57/100
560/560 [==============================] - 0s 635us/sample - loss: 0.0270 - accuracy: 0.9929 - val_loss: 1.6368 - val_accuracy: 0.7000
Epoch 58/100
560/560 [==============================] - 0s 634us/sample - loss: 0.0174 - accuracy: 0.9964 - val_loss: 1.4062 - val_accuracy: 0.7667
Epoch 59/100
560/560 [==============================] - 0s 634us/sample - loss: 0.0210 - accuracy: 0.9946 - val_loss: 1.4429 - val_accuracy: 0.7333
Epoch 60/100
560/560 [==============================] - 0s 634us/sample - loss: 0.0200 - accuracy: 0.9911 - val_loss: 1.4077 - val_accuracy: 0.7500
Epoch 61/100
560/560 [==============================] - 0s 634us/sample - loss: 0.0112 - accuracy: 0.9964 - val_loss: 1.3462 - val_accuracy: 0.7667
Epoch 62/100
560/560 [==============================] - 0s 634us/sample - loss: 0.0191 - accuracy: 0.9964 - val_loss: 1.5998 

Train on 560 samples, validate on 60 samples
Epoch 1/100
560/560 [==============================] - 18s 33ms/sample - loss: 1.0092 - accuracy: 0.6446 - val_loss: 1.8235 - val_accuracy: 0.7667
Epoch 2/100
560/560 [==============================] - 0s 647us/sample - loss: 0.6361 - accuracy: 0.8339 - val_loss: 1.7900 - val_accuracy: 0.7667
Epoch 3/100
560/560 [==============================] - 0s 632us/sample - loss: 0.5247 - accuracy: 0.8518 - val_loss: 0.8742 - val_accuracy: 0.8333
Epoch 4/100
560/560 [==============================] - 0s 642us/sample - loss: 0.4675 - accuracy: 0.8571 - val_loss: 0.5869 - val_accuracy: 0.8833
Epoch 5/100
560/560 [==============================] - 0s 632us/sample - loss: 0.4413 - accuracy: 0.8518 - val_loss: 0.9839 - val_accuracy: 0.7833
Epoch 6/100
560/560 [==============================] - 0s 632us/sample - loss: 0.4150 - accuracy: 0.8661 - val_loss: 0.9439 - val_accuracy: 0.7167
Epoch 7/100
560/560 [==============================] - 0s 632us/sample - 

560/560 [==============================] - 0s 632us/sample - loss: 0.0276 - accuracy: 0.9893 - val_loss: 0.9158 - val_accuracy: 0.7667
Epoch 57/100
560/560 [==============================] - 0s 630us/sample - loss: 0.0233 - accuracy: 0.9946 - val_loss: 1.0975 - val_accuracy: 0.8167
Epoch 58/100
560/560 [==============================] - 0s 632us/sample - loss: 0.0219 - accuracy: 0.9893 - val_loss: 1.0286 - val_accuracy: 0.8333
Epoch 59/100
560/560 [==============================] - 0s 633us/sample - loss: 0.0335 - accuracy: 0.9875 - val_loss: 0.8941 - val_accuracy: 0.8167
Epoch 60/100
560/560 [==============================] - 0s 632us/sample - loss: 0.0270 - accuracy: 0.9893 - val_loss: 0.9775 - val_accuracy: 0.8333
Epoch 61/100
560/560 [==============================] - 0s 633us/sample - loss: 0.0126 - accuracy: 0.9982 - val_loss: 0.8844 - val_accuracy: 0.8500
Epoch 62/100
560/560 [==============================] - 0s 631us/sample - loss: 0.0365 - accuracy: 0.9875 - val_loss: 0.8818 

In [ ]:
import pickle
pickle.dump(best_DNN, open("SCC_DNN_model.pickle.dat", "wb"))

In [10]:
bs = ((ho_vcf.shape[0])/40)
bs = round(bs)
ho_pheno = mlb.transform(ho_pheno)
_, accuracy = best_DNN.evaluate(ho_vcf, ho_pheno, batch_size=bs, verbose=0)
print("Holdout accuracy is " + str(accuracy*100))

Holdout accuracy is 89.03225660324097


In [ ]:
#read in from file
#boy did this get messy lmao
my_list = []
#import label and one hotencoders
le = LabelEncoder()
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
imp = SimpleImputer(missing_values='./.', strategy='most_frequent')
#reads it in chunks
x=0
flag = 0
#drop half the data for memory problems >:(
for chunk in pd.read_csv("SCC_filtered_5pcnt.csv", chunksize=500, index_col="Unnamed: 0"):
    x=x+500
    chunk = chunk.T
    #checks if its the last chunk as Value is the last column
    if 'Value' in chunk.columns:
        #does the selecting of pheno array for application ML
        chunk["Value"] = pd.to_numeric(chunk["Value"], downcast="float")
        pheno = chunk["Value"].to_numpy()
        #reshapes it so its not a 1D array
        print(pheno.shape)
        pheno = np.reshape(pheno,(len(pheno),1))
        print(pheno.shape)
        chunk = chunk.drop(columns=['Value'])
    #applies label and OHE to each chunk, won't include Value column as it'd been dropped
    headers = chunk.columns
    row_idx = chunk.index
    chunk = imp.fit_transform(chunk) #SHOULD TURN ./. into the most common for each column
    chunk = pd.DataFrame(data = chunk, index = row_idx, columns = headers)
    chunk = chunk.apply(lambda col: le.fit_transform(col))
    chunk = ohe.fit_transform(chunk)
    print(chunk.shape)
    if(flag==0):
        my_list.append(chunk)
        print(str(x))
        flag = 1
    else:
        flag = 0
#concats the chunks back in to prevent pandas having a heart attack
#print(pheno.shape)
#my_list.append(pheno)
vcf = np.concatenate(my_list, axis = 1)
print(vcf)
print(pheno)

In [ ]:
#converts the values into a onehot encoded array of each output
#eg. a range of 0,1,2,3 for SCC is converted into 1,0,0,0  0,1,0,0  0,0,1,0  0,0,0,1
mlb = MultiLabelBinarizer()
pheno = mlb.fit_transform(pheno)
print(pheno)
print(pheno.shape)

In [ ]:
#divide up the training and testing data here
X_train, X_test, y_train, y_test = train_test_split(vcf, pheno, test_size=0.2, random_state=27022013)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#My own DNN model based upon paper
#del model #incase its stored a previous model
#del history #for redoing shit

#do batch size as 64
#reduce the inputs by half when you read it in
#add XGboost and RF to the one notebook
def DNN_model():
    model = Sequential()

    #add first input layer, with no normalization
    model.add(Dense(100, input_dim = X_train.shape[1], kernel_initializer='glorot_normal', activity_regularizer=l1(0.0001)))
    model.add(Activation('relu'))

    #add 21 hidden layers with l2 regularization and batch Normalization before activation
    i = 0
    while(i < 21):
            model.add(Dense(50, kernel_initializer='glorot_normal', activity_regularizer=l2(0.0001)))
            model.add(BatchNormalization())
            model.add(Activation('relu'))
            i = i + 1

    #add output layer
    model.add(Dense(4, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=250, batch_size=64)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
_, accuracy = model.evaluate(X_test, y_test, batch_size=64, verbose=0)
print(accuracy)

In [ ]:
vcf = vcf.reshape(vcf.shape[0], vcf.shape[1])
pheno = pheno.reshape(pheno.shape[0], pheno.shape[1])
estimator = KerasClassifier(build_fn=DNN_model, epochs=200, batch_size=64, verbose=1)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, vcf, pheno, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))